In [30]:
import cv2
import pypylon.pylon as py
import numpy as np
import matplotlib.pyplot as plt

# handle exception trace for debugging 
# background loop
import traceback

import time
import random

In [31]:
cam = py.InstantCamera(py.TlFactory.GetInstance().CreateFirstDevice())
cam.Open()

# to get consistant results it is always good to start from "power-on" state
cam.UserSetSelector.Value = "Default"
cam.UserSetLoad.Execute()

cam.ExposureTime.Value = cam.ExposureTime.Min

In [4]:
cam.ExposureTime.Min

19.0

In [5]:
# show expected framerate max framerate ( @ minimum exposure time)
cam.ResultingFrameRate.Value

156.25

In [6]:
# this results in frame period in µs
1 / cam.ResultingFrameRate.Value * 1e6

6400.0

we now compare different pylon grab strategies and their resulting frameperiod to this minimum frameperiod

# Grab scenarios

## GrabOne Loop

most simple style to grab

In [72]:
def GrabOneSample():
    # fetch some images with foreground loop
    img_sum = np.zeros((cam.Height.Value, cam.Width.Value), dtype=np.uint16)

    for i in range(100):
        with cam.GrabOne(1000) as res:
            img = res.Array
            img_sum += img
    return img_sum

In [ ]:
%%timeit -o
GrabOneSample()

In [16]:
grab_one_average = _.average / 100

In [24]:
f"time to capture one frame is: {grab_one_average*1e6:.2f} µs"

'time to capture one frame is: 151589.75 µs'

this is very easy to use, but will have the overhead of starting and stopping the grab engine for every frame

## Foreground Loop

move the start and stop of the grab engine out of the inner grab loop

In [7]:
def ForegroundLoopSample():
    # fetch some images with foreground loop
    img_sum = np.zeros((cam.Height.Value, cam.Width.Value), dtype=np.uint16)
    cam.StartGrabbingMax(100)
    while cam.IsGrabbing():
        with cam.RetrieveResult(1000) as res:
            if res.GrabSucceeded():
                img = res.Array
                img_sum += img
            else:
                raise RuntimeError("Grab failed")
    cam.StopGrabbing()
    return img_sum

In [10]:
img_sum = ForegroundLoopSample()

(1920,)

In [12]:
import matplotlib.pyplot as plt


In [26]:
%%timeit -o 
ForegroundLoopSample()

713 ms ± 61.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 713 ms ± 61.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)>

In [27]:
foreground_average = _.average / 100

In [28]:
f"average time to capture one frame is: {foreground_average*1e6:.2f} µs"

'average time to capture one frame is: 7125.70 µs'

## Background loop

* use this mode to communicate with the camera while the images are collected in the backgound.
* Allows async communication

Pylon uses classes with callback handler to let usercode communicate with the grab loop

In [29]:
class ImageHandler(py.ImageEventHandler):
    def __init__(self):
        super().__init__()
        self.img_sum = np.zeros((cam.Height.Value, cam.Width.Value), dtype=np.uint16)
        
    def OnImageGrabbed(self, camera, grabResult):
        """ we get called on every image
            !! this code is run in a pylon thread context
            always wrap your code in the try .. except to capture
            errors inside the grabbing as this can't be properly reported from 
            the background thread to the foreground python code
        """
        try:
            if grabResult.GrabSucceeded():
                # check image contents
                img = grabResult.Array
                self.img_sum += img
            else:
                raise RuntimeError("Grab Failed")
        except Exception as e:
            traceback.print_exc()

In [37]:
def BackGroundLoopSample():

    # instantiate callback handler
    handler = ImageHandler()
    # register with the pylon loop
    cam.RegisterImageEventHandler(handler , py.RegistrationMode_ReplaceAll, py.Cleanup_None)

    # fetch some images with background loop
    cam.StartGrabbingMax(100, py.GrabStrategy_LatestImages, py.GrabLoop_ProvidedByInstantCamera)
    while cam.IsGrabbing():
        # random exposuretime changes every 100ms
        # cam.ExposureTime.Value = random.uniform(cam.ExposureTime.Min, 1000)
        time.sleep(0.1)

    cam.StopGrabbing()
    cam.DeregisterImageEventHandler(handler)

    return handler.img_sum

In [38]:
%%timeit -o
BackGroundLoopSample()

711 ms ± 443 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 711 ms ± 443 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)>

In [40]:
background_average = _.average / 100

In [41]:
f"average time to capture one frame is: {background_average*1e6:.2f} µs"

'average time to capture one frame is: 7105.98 µs'

In [ ]:
cam.StopGrabbing()

In [9]:
cam.Close()

In [56]:
cam.DeviceTemperatureSelector.SetValue("Coreboard")
print(cam.DeviceTemperature.Value)
cam.BslSensorOn.Execute()
print(cam.BslSensorState.Value)
cam.DeviceTemperatureSelector.SetValue("Sensor")
print(cam.DeviceTemperature.Value)
cam.BslSensorStandby.Execute()
print(cam.BslSensorState.Value)

65.875
On
58.74400000000003
Standby


67.25

In [66]:
time.time() * 1000000

1756233142039616.0

50000
